In [18]:
import pandas as pd
import numpy as np
import warnings
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import Union, List
#import ISLP

warnings.filterwarnings(action='ignore')

In [11]:
class Preprocessor:
    def __init__(self):
        self.file_path = ""
        self.folder_path = ""
        self.df = pd.DataFrame()
    
    def __init__(self, data_file_path:str="", folder_path:str=""):
        self.file_path = data_file_path
        self.folder_path = folder_path
        self.df = pd.DataFrame()
        
    def load_origin_file(self, file_path:str)->None:
        if file_path!="":
            self.file_path = file_path
        self.df = pd.read_csv(self.file_path)
        
    def drop_columns(self, drop_columns_file_path:str = "drop_columns_0410.txt")->None:
        with open(drop_columns_file_path, mode='r') as f:
            drop_fields = f.readlines()
            drop_fields = [drop_field.strip('\n') for drop_field in drop_fields]
        self.df.drop(columns=drop_fields, inplace=True)
    
    def __preprocess_target_variable(self, target_variable:str="loan_status")->None:
        # loan_status가 "current", "issued", "policy" 인 행을 필터링하여 삭제
        modified_df = self.df[~self.df[target_variable].isin(['Current', 'Issued', 'Does not meet the credit policy. Status:Fully Paid', 'Does not meet the credit policy. Status:Charged Off'])]
        # risk = 1, safe = 0 으로 처리
        modified_df.loc[modified_df['loan_status'].isin(['Fully Paid', 'In Grace Period']), 'loan_status'] = 0
        modified_df.loc[modified_df['loan_status'].isin(['Charged Off', 'Default', 'Late (16-30 days)', 'Late (31-120 days)']), 'loan_status'] = 1
        modified_df['loan_status'] = modified_df['loan_status'].astype('int')
        self.df = modified_df
        
    ## 5. 데이터 처리용 함수
    def __delete_suffix(self, term:str)->int:
        '''첫 단어만을 저장하는 함수'''
        term = term.strip().split()[0]
        return int(term)

    def __delete_suffix_percentage(self, term:str)->float:
        '''%를 자르는 함수'''
        term = term.strip('%')
        return float(term)
    
    def __fill_na_with_value(self, columns:List[str], filling_value:Union[str, int])->None:
        '''
        df: dataframe to fill NA
        column_name : column name to change NA values
        filling_value : value type or just value to fill column's NA
        '''
        for column_name in columns:
            if filling_value=="mode":
                mode_value = self.df[column_name].mode()[0]
            elif filling_value=="median":
                mode_value = self.df[column_name].median()
            else:
                mode_value = filling_value
            self.df[column_name].fillna(mode_value, inplace=True)
        
    def __preprocessing_na(self)->None:
        '''
        'acc_open_past_24mths', 확인필요
        'avg_cur_bal', 확인필요
        '''
        ## 결측 처리
        # 결측 개수가 1천 건 이하인 경우는 해당 데이터(row) 삭제
        self.df.dropna(subset=['chargeoff_within_12_mths','collections_12_mths_ex_med','dti',
                                                'pub_rec_bankruptcies','revol_util','tax_liens'], inplace=True)
        
        # A1. 최빈값 대체
        self.__fill_na_with_value(columns=['mo_sin_old_il_acct', 'mths_since_recent_bc', 'mths_since_recent_inq', 'emp_length'], filling_value='mode')
        
        # A2. 중앙값 대체
        self.__fill_na_with_value(columns=['bc_open_to_buy'], filling_value='median')
        # B. 2015년 대체
        # is_after_2015 컬럼 생성. all_util 변수를 기준으로 사용
        self.df['is_after_2015'] = self.df['all_util'].apply(lambda x: 0 if pd.isnull(x) else 1)
        # 결측값을 0으로 채우기
        
        # C. 2012년 대체
        # is_after_2012 컬럼 생성. pct_tl_nvr_dlq 변수를 기준으로 사용
        self.df['is_after_2012'] = self.df['pct_tl_nvr_dlq'].apply(lambda x: 0 if pd.isnull(x) else 1)
        # D. 결측 0 대체
        self.__fill_na_with_value(columns=['annual_inc_joint','dti_joint','revol_bal_joint', 'open_acc_6m',
                                           'open_act_il', 'open_il_12m', 'open_il_24m', 'total_bal_il',
                                           'open_rv_12m', 'open_rv_24m', 'max_bal_bc', 'all_util', 'total_cu_tl', 'mths_since_rcnt_il',
                                           'tot_cur_bal', 'total_rev_hi_lim', 'mo_sin_old_rev_tl_op',
                                           'mo_sin_rcnt_rev_tl_op', 'mo_sin_rcnt_tl', 'mort_acc', 'num_bc_sats', 'num_bc_tl',
                                           'num_accts_ever_120_pd', 'num_actv_bc_tl', 'num_actv_rev_tl', 'num_il_tl',
                                           'num_op_rev_tl','num_rev_accts','num_rev_tl_bal_gt_0','num_sats','num_tl_120dpd_2m','num_tl_30dpd',
                                           'num_tl_90g_dpd_24m','num_tl_op_past_12m','pct_tl_nvr_dlq','tot_hi_cred_lim','total_bal_ex_mort',
                                           'total_bc_limit','total_il_high_credit_limit'], filling_value=0)
        
        
    def __convert_object_to_numeric(self, column_name:str)->pd.DataFrame:
        unique_values = sorted(self.df[column_name].unique())
        value_map = {value:index for index, value in enumerate(unique_values)}
        self.df[column_name] = self.df[column_name].apply(lambda x:value_map.get(x))
        return self.df
    
    def __convert_object_to_one_hot(self, column_name:str)->None:
        encoded = pd.get_dummies(self.df[column_name])
        self.df = pd.concat([self.df, encoded], axis=1)
        self.df.drop(column_name, axis=1, inplace=True)
        
    def __preprocessing_objects(self)->None:
        ## TODO : 'addr_state' 필드 해결하기
        # term
        self.df['term'] = self.df['term'].apply(self.__delete_suffix)
        # emp_length
        self.df['emp_length'] = self.df['emp_length'].apply(lambda x: x.replace(' years','').replace(' year','').replace('+','').replace('< 1', '0'))
        self.df['emp_length'] = self.df['emp_length'].astype(int)
        # revol_util
        self.df['revol_util'] = self.df['revol_util'].apply(self.__delete_suffix_percentage)
        
        ## numeric
        # application_type
        self.df = self.__convert_object_to_numeric('application_type')
        # sub_grade
        self.df = self.__convert_object_to_numeric('sub_grade')

        ## one-hot
        # home_ownership
        self.df['home_ownership'] = self.df['home_ownership'].replace(['ANY', 'OTHER', 'NONE'], 'OTHERS')
        self.__convert_object_to_one_hot('home_ownership')
        # purpose
        self.__convert_object_to_one_hot('purpose')
        # verification_status
        self.__convert_object_to_one_hot('verification_status')
        # addr_state : 해야함...

    def preprocess(self)->None:
        # loan_status 제외 모든 column이 결측치(na)인 행 제거 (1개 행 제거됨)
        self.df.dropna(subset=self.df.columns.difference(['loan_status']),how='all', inplace=True)
        self.__preprocess_target_variable()
        # 결측치 제거
        self.__preprocessing_na()
        ## object 처리하기
        self.__preprocessing_objects()
        # index 재설정
        self.df.reset_index(drop=True, inplace=True)
        self.df.dropna(subset=self.df.columns.difference(['loan_status']),how='all', inplace=True)
        
    def get_df(self)->pd.DataFrame:
        return self.df


p = Preprocessor()
p.load_origin_file(file_path="lending_club_2020_train.csv")
p.drop_columns(drop_columns_file_path='drop_columns_0410.txt')
p.preprocess()
df = p.get_df()
columns_with_na = df.columns[df.isna().any()].tolist()

In [12]:
df = df.drop(columns=['addr_state','acc_open_past_24mths', 'avg_cur_bal'])
X = df.drop(columns=['loan_status'])

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, df['loan_status'], test_size=0.2, random_state=42)

## RidgeClassifier

In [20]:
from sklearn.linear_model import RidgeClassifierCV
clf = RidgeClassifierCV().fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.52      0.64    180516
           1       0.24      0.60      0.34     45821

    accuracy                           0.54    226337
   macro avg       0.54      0.56      0.49    226337
weighted avg       0.72      0.54      0.58    226337



## LassoClassifier

In [25]:
from sklearn.linear_model import LogisticRegressionCV
clf = LogisticRegressionCV(penalty='l1', solver = 'saga').fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))
#48분 걸려서 recall 0!

              precision    recall  f1-score   support

           0       0.80      1.00      0.89    180516
           1       0.37      0.00      0.00     45821

    accuracy                           0.80    226337
   macro avg       0.58      0.50      0.44    226337
weighted avg       0.71      0.80      0.71    226337



## LDA, QDA

In [21]:
#LDA
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
clf = LinearDiscriminantAnalysis().fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.81      0.98      0.89    180516
           1       0.51      0.10      0.16     45821

    accuracy                           0.80    226337
   macro avg       0.66      0.54      0.52    226337
weighted avg       0.75      0.80      0.74    226337



In [22]:
# QDA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
clf = QuadraticDiscriminantAnalysis().fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.70      0.77    180516
           1       0.30      0.52      0.38     45821

    accuracy                           0.66    226337
   macro avg       0.58      0.61      0.57    226337
weighted avg       0.74      0.66      0.69    226337



## Naive Bayes

In [23]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB().fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.84      0.81      0.82    180516
           1       0.35      0.41      0.37     45821

    accuracy                           0.73    226337
   macro avg       0.59      0.61      0.60    226337
weighted avg       0.74      0.73      0.73    226337



## SVM

In [26]:
from sklearn import svm
clf = svm.SVC()
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

: 

: 